In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from modules.BatchPitNormalization import BatchPitNorm1d
import torch.nn as nn 
import torch.optim as optim

device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using {device} device")


Using cuda device


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

data = np.genfromtxt('data/housing.csv', delimiter=",",skip_header=1)

data = np.delete(data, [-1], axis=1)
scaler = StandardScaler()

X = data[:,:8]
Y = data[:,8].reshape(-1,1)



Y=Y[~np.isnan(X).any(axis=1)]
X=X[~np.isnan(X).any(axis=1)]


X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(X, Y, train_size = 0.7, shuffle = True)

scaler.fit(X_train_raw)
X_train = scaler.transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

scaler.fit(y_train_raw)
y_train = scaler.transform(y_train_raw)
y_test = scaler.transform(y_test_raw)

X_train = torch.tensor(X_train, dtype=torch.float32, device=device)
y_train = torch.tensor(y_train, dtype=torch.float32, device=device).reshape(-1,1)
X_test = torch.tensor(X_test, dtype=torch.float32, device=device)
y_test = torch.tensor(y_test, dtype=torch.float32, device=device).reshape(-1,1)



In [4]:
class CalHouse(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(8, 24)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(24, 12)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(12, 1)
        #self.act_output = nn.Sigmoid()
 
    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        #x = self.act_output(self.output(x))
        return x
 
model = CalHouse()
model = model.cuda()
print(model)

CalHouse(
  (hidden1): Linear(in_features=8, out_features=24, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=24, out_features=12, bias=True)
  (act2): ReLU()
  (output): Linear(in_features=12, out_features=1, bias=True)
)


In [5]:
# Define the model

model = nn.Sequential(
    nn.Linear(8, 24),
    nn.ReLU(),
    nn.Linear(24, 12),
    nn.ReLU(),
    nn.Linear(12, 6),
    nn.ReLU(),
    nn.Linear(6, 1)
)
model.cuda()
print(model)

Sequential(
  (0): Linear(in_features=8, out_features=24, bias=True)
  (1): ReLU()
  (2): Linear(in_features=24, out_features=12, bias=True)
  (3): ReLU()
  (4): Linear(in_features=12, out_features=6, bias=True)
  (5): ReLU()
  (6): Linear(in_features=6, out_features=1, bias=True)
)


In [6]:
lossFunction = nn.MSELoss()  
optimizer = optim.Adam(model.parameters(), lr = 0.001)

n_epochs = 100
batch_size = 20
 
for epoch in range(n_epochs):
    for i in range(0, len(X_train), batch_size):
        Xbatch = X_train[i:i+batch_size]
        ybatch = y_train[i:i+batch_size]
        y_pred = model(Xbatch)
        loss = lossFunction(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')




Finished epoch 0, latest loss 0.18365320563316345
Finished epoch 1, latest loss 0.16407924890518188
Finished epoch 2, latest loss 0.1407858431339264
Finished epoch 3, latest loss 0.11659590899944305
Finished epoch 4, latest loss 0.10625474154949188
Finished epoch 5, latest loss 0.09919671714305878
Finished epoch 6, latest loss 0.09108760952949524
Finished epoch 7, latest loss 0.08359266817569733
Finished epoch 8, latest loss 0.07961805909872055
Finished epoch 9, latest loss 0.08113346993923187
Finished epoch 10, latest loss 0.08316367119550705
Finished epoch 11, latest loss 0.08363928645849228
Finished epoch 12, latest loss 0.07377368211746216
Finished epoch 13, latest loss 0.07563924789428711
Finished epoch 14, latest loss 0.07098429650068283
Finished epoch 15, latest loss 0.07403436303138733
Finished epoch 16, latest loss 0.076614610850811
Finished epoch 17, latest loss 0.0718265250325203
Finished epoch 18, latest loss 0.07504728436470032
Finished epoch 19, latest loss 0.070484094321